In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import sklearn.decomposition as decomposition
from sklearn.manifold import TSNE

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import seaborn as sb
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches

In [2]:
df_train = pd.read_csv("data/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("data/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

n_class = np.unique(Y).size

X.shape, Y.shape

((87554, 187, 1), (87554,))

In [15]:
seq_len = 187

def get_model():
    n_class = 5
    model = Sequential()
    model.add(LSTM(128, 
                   input_shape=(seq_len, 1), 
                   activation='relu', 
                   return_sequences=True,
                   dropout=0.2))
    model.add(LSTM(128, 
                   activation='relu',
                   return_sequences=False, 
                   dropout=0.1))
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(n_class, activation='softmax'))
    
    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
    
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'],
    )
    return model

def get_simple_model():
    n_class = 5
    model = Sequential()
    model.add(LSTM(32, 
                   input_shape=(seq_len, 1), 
                   activation='relu', 
                   return_sequences=False,
                   dropout=0.2))
    
    model.add(Dense(32, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(n_class, activation='softmax'))
    
    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
    
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'],
    )
    return model

In [16]:
model = get_simple_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 165       
Total params: 5,573
Trainable params: 5,573
Non-trainable params: 0
_________________________________________________________________


In [10]:
mini_X = X[:1000, :, :]
mini_Y = Y[:1000]

np.unique(mini_Y, return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int8),
 array([848,  17,  58,  10,  67], dtype=int64))

In [17]:
model.fit(X[:1000, :, :], Y[:1000], 
          epochs=2, 
          verbose=1,
          batch_size=32,
          validation_split=0.1)

Train on 900 samples, validate on 100 samples
Epoch 1/2
900/900 [==============================] - 13s 15ms/sample - loss: 1.5165 - accuracy: 0.8489 - val_loss: 1.3513 - val_accuracy: 0.8300
Epoch 2/2
900/900 [==============================] - 10s 11ms/sample - loss: nan - accuracy: 0.8500 - val_loss: nan - val_accuracy: 0.8300


In [11]:
model.predict(mini_X)

array([[0.305817  , 0.15159464, 0.1644598 , 0.19375268, 0.18437594],
       [0.30581692, 0.15159465, 0.16445978, 0.19375265, 0.18437594],
       [0.30581677, 0.15159471, 0.16445987, 0.19375265, 0.18437597],
       ...,
       [0.30581686, 0.15159468, 0.16445982, 0.19375265, 0.18437594],
       [0.3058147 , 0.1515957 , 0.16446076, 0.19375263, 0.18437621],
       [0.30581692, 0.15159465, 0.16445978, 0.19375265, 0.18437594]],
      dtype=float32)